# ElephantSQL with PostgresSQL2

This notebook is some cursory notes on setting up Databases and creating tables. It is using an environment where the pymongo and psycopg2 are installed. 

## Connecting 

In [1]:
# First I'll be working with a relational database. 
# It is hosted by ElephantSQL
## I'll be using PostgreSQL2
## http://initd.org/psycopg/

import psycopg2

In [2]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'connect',
 'extensions',
 'paramstyle',
 'threadsafety',
 'tz']

The method I am looking for is "connect". 

In [3]:
help(psycopg2.connect)

Help on function connect in module psycopg2:

connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)
    Create a new database connection.
    
    The connection parameters can be specified as a string:
    
        conn = psycopg2.connect("dbname=test user=postgres password=secret")
    
    or using a set of keyword arguments:
    
        conn = psycopg2.connect(database="test", user="postgres", password="secret")
    
    Or as a mix of both. The basic connection parameters are:
    
    - *dbname*: the database name
    - *database*: the database name (only as keyword argument)
    - *user*: user name used to authenticate
    - *password*: password used to authenticate
    - *host*: database host address (defaults to UNIX socket if not provided)
    - *port*: connection port number (defaults to 5432 if not provided)
    
    Using the *connection_factory* parameter a different class or connections
    factory can be specified. It should be a callable object tak

This is great info. The specific information I need is :

     The connection parameters can be specified as a string:
     conn = psycopg2.connect("dbname=test user=postgres password=secret")
     
 Where the parameters are:
 
    - *dbname*: the database name
    - *database*: the database name (only as keyword argument)
    - *user*: user name used to authenticate
    - *password*: password used to authenticate
    - *host*: database host address (defaults to UNIX socket if not provided)
    - *port*: connection port number (defaults to 5432 if not provided)
    
For the current playground database, I don;t need *database* and *port*

In [33]:
# Establishing Parameters
dbname = 'ckdaorek'
user = 'ckdaorek'
password = '******'   # Changed so you can't see my password. 
host = 'raja.db.elephantsql.com'

In [34]:
# Creating a connection to my database
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [35]:
# Connection cretes a connection object containing my paremeters
pg_conn

<connection object at 0x10eac17f8; dsn: 'user=ckdaorek password=xxx dbname=ckdaorek host=raja.db.elephantsql.com', closed: 0>

In [39]:
# Creating my cursor
pg_curs = pg_conn.cursor()

## Create a Table on ElephantSQL from rpg_db
Now I am going to use **sqlite** to import the **rpg_db** and upload it to the connected database. 

In [14]:
import sqlite3

# I'm using a s1 connection and cursor so I don't overlap databases and libraries. 
sl_conn = sqlite3.connect('/Users/azel/Documents/GitHub/DS-Unit-3-Sprint-2-SQL-and-Databases/module1-introduction-to-sql/rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [15]:
# Quick test to make sure its working and the database is populated. 
sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

[(302,)]

In [24]:
# Each row is represented as a tuple. Those tuples can be accessed by an integer index. 
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()
characters [0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [25]:
# Each part of the tuple can be indexed as well. 
characters [0][1]

'Aliquid iste optio reiciendi'

So to create a table on my Relational database, I need to provide the columns and their datatypes. This metadata will match up with the indices of each row-tuple. This is the information provided with the rpg_db:

```python
class Character(models.Model):
    """Base representation of RPG characters."""
    character_id = models.AutoField(primary_key=True)
    name = models.CharField("Character's name", max_length=30)
    level = models.IntegerField(default=0)
    exp = models.IntegerField(default=0)
    hp = models.IntegerField(default=10)
    strength = models.IntegerField(default=1)
    intelligence = models.IntegerField(default=1)
    dexterity = models.IntegerField(default=1)
    wisdom = models.IntegerField(default=1)
```

With this, I can create a table in the database with these columns and the required datatype. 

In [40]:
create_character_table = """
CREATE TABLE charactercreator_character (
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
)
"""

In [43]:
pg_conn.commit() 

In [44]:
pg_curs.execute(create_character_table)

In [45]:
attempted_insert = """
  INSERT INTO charactercreator_character
  VALUES """ + str(characters[0])
attempted_insert

"\n  INSERT INTO charactercreator_character\n  VALUES (1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [46]:
pg_curs.execute(attempted_insert)

In [47]:
pg_conn.commit()  # We need to "save" our work for the new table/data to show up

In [48]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

Ok, that works! I am going to insert the remaining characters into the database. 

In [49]:
# Let's insert the rest of the characters!
for character in characters[1:]:
  insert_character = """
  INSERT INTO charactercreator_character
  VALUES """ + str(character)
  pg_curs.execute(insert_character)
pg_conn.commit()

In [50]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [51]:
len(pg_characters)

302

In [52]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [53]:
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character

Nothing happened! That's a good thing since the `assert` function will return an error if the statement is not `True`.

This is how to push a create a relational database in the cloud. The next step is to take a csv and move it up as well.

## Titanic Dataset to ElephantSQL

In [131]:
import pandas as pd
df = pd.read_csv('/Users/azel/Documents/GitHub/DS-Unit-3-Sprint-2-SQL-and-Databases/module2-sql-for-analysis/titanic.csv')

# MongoDB with PyMongo

LSDS Unit 3 Sprint 2 Module 3

Some resources:

https://docs.atlas.mongodb.com/getting-started/

https://api.mongodb.com/python/current/

HN Discussion on MongoDB versus PostgreSQL/SQLite: https://news.ycombinator.com/item?id=19158854

## Connecting

In [54]:
import pymongo

For some reason, pymongo was throwing an error when I used the link designed for Python 3.6. It seems the link designed for Python 3.4 works better. 

In [55]:
# Password removed before committing. 
client = pymongo.MongoClient("mongodb://admin:<password>@cluster0-shard-00-00-l0vzq.mongodb.net:27017,cluster0-shard-00-01-l0vzq.mongodb.net:27017,cluster0-shard-00-02-l0vzq.mongodb.net:27017/test?ssl=true&replicaSet=Cluster0-shard-0&authSource=admin&retryWrites=true&w=majority")
db = client.test

In [58]:
print('DB info: \n',db, '\n\n')
print('My nodes: \n', client.nodes)

DB info: 
 Database(MongoClient(host=['cluster0-shard-00-01-l0vzq.mongodb.net:27017', 'cluster0-shard-00-02-l0vzq.mongodb.net:27017', 'cluster0-shard-00-00-l0vzq.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, replicaset='Cluster0-shard-0', authsource='admin', retrywrites=True, w='majority'), 'test') 


My nodes: 
 frozenset({('cluster0-shard-00-01-l0vzq.mongodb.net', 27017), ('cluster0-shard-00-02-l0vzq.mongodb.net', 27017), ('cluster0-shard-00-00-l0vzq.mongodb.net', 27017)})


I'm connected and have 3 nodes available. Time to start adding data. 

## Add Data to a MongoDB Database

In [70]:
# I want to see what I can do. 
help(db)

Help on Database in module pymongo.database object:

class Database(pymongo.common.BaseObject)
 |  Database(client, name, codec_options=None, read_preference=None, write_concern=None, read_concern=None)
 |  
 |  A Mongo database.
 |  
 |  Method resolution order:
 |      Database
 |      pymongo.common.BaseObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      This is only here so that some API misusages are easier to debug.
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getattr__(self, name)
 |      Get a collection of this database by name.
 |      
 |      Raises InvalidName if an invalid collection name is used.
 |      
 |      :Parameters:
 |        - `name`: the name of the collection to get
 |  
 |  __getitem__(self, name)
 |      Get a collection of this database by name.
 |      
 |      Raises InvalidName if an invalid collection name is used.
 |      
 |      :Parameters:
 |        - `name`: the na

In [71]:
# This command refers to "test" collection on my cluster.
dir(db.test)

['_BaseObject__codec_options',
 '_BaseObject__read_concern',
 '_BaseObject__read_preference',
 '_BaseObject__write_concern',
 '_Collection__create',
 '_Collection__create_index',
 '_Collection__database',
 '_Collection__find_and_modify',
 '_Collection__full_name',
 '_Collection__name',
 '_Collection__write_response_codec_options',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_aggregate',
 '_aggregate_one_result',
 '_command',
 '_count',
 '_delete',
 '_delete_retryable',
 '_insert',
 '_insert_one',
 '_legacy_write',
 '_read_preference_for',
 '_socket_for_primary_reads',
 '_socket_for_reads',
 '_s

So I have the `db object` as `db` and the specefic collection (table-like) as `test` for my method. I could call `rpg_db` as the method if I want to modify that collection. I'll be doing that later. 

In [72]:
# Lets get information on the insert_one method. 
help(db.test.insert_one)

Help on method insert_one in module pymongo.collection:

insert_one(document, bypass_document_validation=False, session=None) method of pymongo.collection.Collection instance
    Insert a single document.
    
      >>> db.test.count_documents({'x': 1})
      0
      >>> result = db.test.insert_one({'x': 1})
      >>> result.inserted_id
      ObjectId('54f112defba522406c9cc208')
      >>> db.test.find_one({'x': 1})
      {u'x': 1, u'_id': ObjectId('54f112defba522406c9cc208')}
    
    :Parameters:
      - `document`: The document to insert. Must be a mutable mapping
        type. If the document does not have an _id field one will be
        added automatically.
      - `bypass_document_validation`: (optional) If ``True``, allows the
        write to opt-out of document level validation. Default is
        ``False``.
      - `session` (optional): a
        :class:`~pymongo.client_session.ClientSession`.
    
    :Returns:
      - An instance of :class:`~pymongo.results.InsertOneResult`

The info provides a nice example as well as important parameters:

     :Parameters:
      - `document`: The document to insert. Must be a mutable mapping
        type. If the document does not have an _id field one will be
        added automatically.
      - `bypass_document_validation`: (optional) If ``True``, allows the
        write to opt-out of document level validation. Default is
        ``False``.
      - `session` (optional): a
        :class:`~pymongo.client_session.ClientSession`.
    
**Mutable mapping type** can be defined as:

>A mapping object maps immutable values to arbitrary objects. Mappings are mutable objects. There is currently only one standard mapping type, the dictionary. A dictionary's keys are almost arbitrary values. Only values containing lists, dictionaries or other mutable types (that are compared by value rather than by object identity) may not be used as keys. Numeric types used for keys obey the normal rules for numeric comparison: if two numbers compare equal (such as 1 and 1.0) then they can be used interchangeably to index the same dictionary entry.

In summary, I should use a dictionary. However, that dictionary's key must be arbitrary and thus, immuatable. In this case, I could use a tuple, but the tuple **cannot contain numeric datatypes**. 

I'm going to test out some isnertions to the `test` collection on my database. 

In [62]:
db.test.insert_one({'x': 1})

In [69]:
#That looks like it worked, but I want to see if I can prove that it did
help(db.test.find_one)

Help on method find_one in module pymongo.collection:

find_one(filter=None, *args, **kwargs) method of pymongo.collection.Collection instance
    Get a single document from the database.
    
    All arguments to :meth:`find` are also valid arguments for
    :meth:`find_one`, although any `limit` argument will be
    ignored. Returns a single document, or ``None`` if no matching
    document is found.
    
    The :meth:`find_one` method obeys the :attr:`read_preference` of
    this :class:`Collection`.
    
    :Parameters:
    
      - `filter` (optional): a dictionary specifying
        the query to be performed OR any other type to be used as
        the value for a query for ``"_id"``.
    
      - `*args` (optional): any additional positional arguments
        are the same as the arguments to :meth:`find`.
    
      - `**kwargs` (optional): any additional keyword arguments
        are the same as the arguments to :meth:`find`.
    
          >>> collection.find_one(max_time_ms=

In [65]:
# Finds one instance where the document matches the parameter dictionary
db.test.find_one({'x': 1})

{'_id': ObjectId('5cf7faecb0e0522fb5285830'), 'x': 1}

In [101]:
# Lets see what kind of methods are available to an object with find attached. 
results = db.test.find({'x': 1})
dir(results)

['_Cursor__address',
 '_Cursor__batch_size',
 '_Cursor__check_okay_to_chain',
 '_Cursor__codec_options',
 '_Cursor__collation',
 '_Cursor__collection',
 '_Cursor__comment',
 '_Cursor__data',
 '_Cursor__die',
 '_Cursor__empty',
 '_Cursor__exhaust',
 '_Cursor__exhaust_mgr',
 '_Cursor__explain',
 '_Cursor__explicit_session',
 '_Cursor__hint',
 '_Cursor__id',
 '_Cursor__killed',
 '_Cursor__limit',
 '_Cursor__manipulate',
 '_Cursor__max',
 '_Cursor__max_await_time_ms',
 '_Cursor__max_scan',
 '_Cursor__max_time_ms',
 '_Cursor__min',
 '_Cursor__modifiers',
 '_Cursor__ordering',
 '_Cursor__projection',
 '_Cursor__query_flags',
 '_Cursor__query_spec',
 '_Cursor__read_concern',
 '_Cursor__read_preference',
 '_Cursor__retrieved',
 '_Cursor__return_key',
 '_Cursor__send_message',
 '_Cursor__session',
 '_Cursor__set_hint',
 '_Cursor__show_record_id',
 '_Cursor__skip',
 '_Cursor__snapshot',
 '_Cursor__spec',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir

In [102]:
# view the results tied up in the find object 
list(results)

[{'_id': ObjectId('5cf7faecb0e0522fb5285830'), 'x': 1},
 {'_id': ObjectId('5cf842ebb0e0523ab9fdcb76'), 'x': 1}]

So the documents can be pulled from the `find` object by turning it into a list. The information can be viewed this way. 

In [67]:
# Let's throw in a few more docmuents. 
artins_doc = {'favorite animal': 'silver panther'}
artins_doc

{'favorite animal': 'silver panther'}

In [68]:
# Insert to the Database. 
db.test.insert_one(artins_doc)

In [77]:
# Does not affect the previously saved variable. 
list(results)

[{'_id': ObjectId('5cf7faecb0e0522fb5285830'), 'x': 1},
 {'_id': ObjectId('5cf842ebb0e0523ab9fdcb76'), 'x': 1}]

In [78]:
# Proof that the new document is there. 
list(db.test.find())

[{'_id': ObjectId('5cf7faecb0e0522fb5285830'), 'x': 1},
 {'_id': ObjectId('5cf842ebb0e0523ab9fdcb76'), 'x': 1},
 {'_id': ObjectId('5cf8431fb0e0523ab9fdcb77'),
  'favorite animal': 'silver panther'}]

In [79]:
# Add a couple more
alexanders_doc = {'favorite animal': 'parrot'}
wills_favorite = {'favorite animal': 'goose'}

In [80]:
help(db.test.insert_many)

Help on method insert_many in module pymongo.collection:

insert_many(documents, ordered=True, bypass_document_validation=False, session=None) method of pymongo.collection.Collection instance
    Insert an iterable of documents.
    
      >>> db.test.count_documents({})
      0
      >>> result = db.test.insert_many([{'x': i} for i in range(2)])
      >>> result.inserted_ids
      [ObjectId('54f113fffba522406c9cc20e'), ObjectId('54f113fffba522406c9cc20f')]
      >>> db.test.count_documents({})
      2
    
    :Parameters:
      - `documents`: A iterable of documents to insert.
      - `ordered` (optional): If ``True`` (the default) documents will be
        inserted on the server serially, in the order provided. If an error
        occurs all remaining inserts are aborted. If ``False``, documents
        will be inserted on the server in arbitrary order, possibly in
        parallel, and all document inserts will be attempted.
      - `bypass_document_validation`: (optional) If ``Tru

In [ ]:
db.test.insert_many([alexanders_doc, wills_favorite])

The insert many method can be used to batch insert documents generated by programs. 

In [83]:
docs = []
for i in range(10):
  doc = {'even': i % 2 == 0}
  doc['value'] = i
  docs.append(doc)

docs

[{'even': True, 'value': 0},
 {'even': False, 'value': 1},
 {'even': True, 'value': 2},
 {'even': False, 'value': 3},
 {'even': True, 'value': 4},
 {'even': False, 'value': 5},
 {'even': True, 'value': 6},
 {'even': False, 'value': 7},
 {'even': True, 'value': 8},
 {'even': False, 'value': 9}]

In [84]:
db.test.insert_many(docs)

However, it is sensitive to improper datatypes. For example, the dictionary below will return an error. Although a tuple is immutable, it wont allow the values in that tuple to be numeric datatypes. 

In [85]:
cool_dict = {
    'awesome': True,
    'even': False,
    'some numbers': [1, 2, 3],
    (3, 4): (5, 6)
}

In [86]:
db.test.insert_one(cool_dict)

InvalidDocument: documents must have only string keys, key was (3, 4)

In [87]:
# Turn those tuples into strings
fixed_dict = {str(key): value for key, value in cool_dict.items()}
fixed_dict

{'awesome': True,
 'even': False,
 'some numbers': [1, 2, 3],
 '(3, 4)': (5, 6),
 '_id': ObjectId('5cf84641b0e0523ab9fdcb84')}

In [88]:
db.test.insert_one(fixed_dict)

Can I insert data in a tuple like with PostgreSQL? 

In [90]:
rpg_character = (1, "King Bob", 10, 3, 0, 0, 0)

In [91]:
db.test.insert_one(rpg_character)

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

Turns out you can't insert information with pymongo like with psycopg2. Therefore, when inserting data, best practice might be to code the feature information into the key of the dictionary. Below are a couple methods on inserting document, one without specific descriptors for the keys, and one with. 

In [92]:
# The key as the identifier. Assumes the user would know the feature information. Could be identified in the first document. 
db.test.insert_one({'rpg_character': rpg_character})

In [93]:
# End result: with unique objectId, rpg_character identifier key and list containing the information associated with the character. 
db.test.find_one({'rpg_character': rpg_character})

{'_id': ObjectId('5cf846c7b0e0523ab9fdcb85'),
 'rpg_character': [1, 'King Bob', 10, 3, 0, 0, 0]}

In [94]:
# Let's insert a new document with more robust keys referring the names of the stats. 
db.test.insert_one({
    'sql_id': 1,
    'name': rpg_character[1],
    'hp': rpg_character[2],
    'level': rpg_character[3]
})

In [95]:
# Lets see the documents. 
list(db.test.find())

[{'_id': ObjectId('5cf7faecb0e0522fb5285830'), 'x': 1},
 {'_id': ObjectId('5cf842ebb0e0523ab9fdcb76'), 'x': 1},
 {'_id': ObjectId('5cf8431fb0e0523ab9fdcb77'),
  'favorite animal': 'silver panther'},
 {'_id': ObjectId('5cf84530b0e0523ab9fdcb78'), 'favorite animal': 'parrot'},
 {'_id': ObjectId('5cf84530b0e0523ab9fdcb79'), 'favorite animal': 'goose'},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb7a'), 'even': True, 'value': 0},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb7b'), 'even': False, 'value': 1},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb7c'), 'even': True, 'value': 2},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb7d'), 'even': False, 'value': 3},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb7e'), 'even': True, 'value': 4},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb7f'), 'even': False, 'value': 5},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb80'), 'even': True, 'value': 6},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb81'), 'even': False, 'value': 7},
 {'_id': ObjectId('5cf84542b0e0523ab9fdcb82'), 'even

In [96]:
# Add another character
another_character = (2, "Queen Jane", 10, 4, 0, 0, 0)
db.test.insert_one({
    'sql_id': 1,
    'name': another_character[1],
    'hp': another_character[2],
    'level': another_character[3]
})

In [98]:
# Use the find method with specific parameters to pull any documents that match the parameters from the database. 
list(db.test.find({'hp': 10, 'level': 4}))

[{'_id': ObjectId('5cf847f0b0e0523ab9fdcb87'),
  'sql_id': 1,
  'name': 'Queen Jane',
  'hp': 10,
  'level': 4}]

## Adding the rpg_db info to the MongoDB cloud

In [110]:
# Let's set the client to the rpg database. I'll be submitting data from the charactercreator_character table from rpg_db.sqlite3
db = client.rpg

In [113]:
# To make sure this works, I'm going to connect to the charactercreator_character collection within the rpg database. 
list(db.charactercreator_character.find())

[]

It works! So I'm going to pull the information from the `rpg_db.sqlite3` file. Specifically, the `charactercreator_character` table. 

In [117]:
# I'm going to pull all the information from the charactercreator_character table.
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()
# for testing purposes, I'm also going to select a single row. 
single_character = characters[0]
# Now to look at the data
characters

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1),
 (6, 'Non nobis et of', 0, 0, 10, 1, 1, 1, 1),
 (7, 'Perferendis', 0, 0, 10, 1, 1, 1, 1),
 (8, 'Accusantium amet quidem eve', 0, 0, 10, 1, 1, 1, 1),
 (9, 'Sed nostrum inventore error m', 0, 0, 10, 1, 1, 1, 1),
 (10, 'Harum repellendus omnis od', 0, 0, 10, 1, 1, 1, 1),
 (11, 'Itaque ut commodi,', 0, 0, 10, 1, 1, 1, 1),
 (12, 'Molestiae quis', 0, 0, 10, 1, 1, 1, 1),
 (13, 'Ali', 0, 0, 10, 1, 1, 1, 1),
 (14, 'Tempora quod optio possimus il', 0, 0, 10, 1, 1, 1, 1),
 (15, 'Sed itaque beatae pari', 0, 0, 10, 1, 1, 1, 1),
 (16, 'Quam dolor', 0, 0, 10, 1, 1, 1, 1),
 (17, 'Molestias expedita', 0, 0, 10, 1, 1, 1, 1),
 (18, 'Lauda', 0, 0, 10, 1, 1, 1, 1),
 (19, 'Incidunt sint perferen', 0, 0, 10, 1, 1, 1, 1),
 (20, 'Laboriosa', 0, 0, 10, 1, 1, 1, 1),
 (21,

In [118]:
single_character

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

So the data is here. Just to look at the key information one more time:

```python
class Character(models.Model):
    """Base representation of RPG characters."""
    character_id = models.AutoField(primary_key=True)
    name = models.CharField("Character's name", max_length=30)
    level = models.IntegerField(default=0)
    exp = models.IntegerField(default=0)
    hp = models.IntegerField(default=10)
    strength = models.IntegerField(default=1)
    intelligence = models.IntegerField(default=1)
    dexterity = models.IntegerField(default=1)
    wisdom = models.IntegerField(default=1)
```

In [122]:
doc = {
    'sql_id' : single_character[0],
    'name' : single_character[1],
    'level' : single_character[2],
    'exp' : single_character[3],
    'hp' : single_character[4],
    'strength' : single_character[5],
    'intelligence' : single_character[6],
    'dexterity' : single_character[7],
    'wisdom' : single_character[8]
}

In [123]:
doc

{'sql_id': 1,
 'name': 'Aliquid iste optio reiciendi',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1}

In [127]:
# Now I can use this template to loop through the character database. 
# I will use a list comprehension, then the insert_many method. 

documents = []
for i in list(range(len(characters))):
    document = {'sql_id' : characters[i][0],
                        'name' : characters[i][1],
                        'level' : characters[i][2],
                        'exp' : characters[i][3],
                        'hp' : characters[i][4],
                        'strength' : characters[i][5],
                        'intelligence' : characters[i][6],
                        'dexterity' : characters[i][7],
                        'wisdom' : characters[i][8]}
    documents.append(document)
    
documents

[{'sql_id': 1,
  'name': 'Aliquid iste optio reiciendi',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'sql_id': 2,
  'name': 'Optio dolorem ex a',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'sql_id': 3,
  'name': 'Minus c',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'sql_id': 4,
  'name': 'Sit ut repr',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'sql_id': 5,
  'name': 'At id recusandae expl',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'sql_id': 6,
  'name': 'Non nobis et of',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'sql_id': 7,
  'name': 'Perferendis',
  'level': 0,
  'exp': 0,


In [129]:
# Good thing the insert_many method takes a list as a parameter!
db.charactercreator_character.insert_many(documents)

In [132]:
list(db.charactercreator_character.find())

[{'_id': ObjectId('5cf882acb0e0523ab9fdcb88'),
  'sql_id': 1,
  'name': 'Aliquid iste optio reiciendi',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'_id': ObjectId('5cf882acb0e0523ab9fdcb89'),
  'sql_id': 2,
  'name': 'Optio dolorem ex a',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'_id': ObjectId('5cf882acb0e0523ab9fdcb8a'),
  'sql_id': 3,
  'name': 'Minus c',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'_id': ObjectId('5cf882acb0e0523ab9fdcb8b'),
  'sql_id': 4,
  'name': 'Sit ut repr',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
  'wisdom': 1},
 {'_id': ObjectId('5cf882acb0e0523ab9fdcb8c'),
  'sql_id': 5,
  'name': 'At id recusandae expl',
  'level': 0,
  'exp': 0,
  'hp': 10,
  'strength': 1,
  'intelligence': 1,
  'dexterity': 1,
 

## Titanic Dataset to MongoDB

In [164]:
# Changing the database that I am working on.
db = client.titanic

df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


I can do a list comprehension to create a list of dictionaries just like the rpg_db. The effect should be the same. 

In [189]:
documents = []

for i in list(range(len(df))):
    document = {
        'Survived' : str(df['Survived'][i]),
        'Pclass' : str(df['Pclass'][i]),
        'Name' : str(df['Name'][i]),
        'Sex' : str(df['Sex'][i]),
        'Age' : str(df['Age'][i]),
        'Siblings_Spouses Aboard' : str(df['Siblings/Spouses Aboard'][i]),
        'Parents_Children Aboard' : str(df['Parents/Children Aboard'][i]),
        'Fare' : str(df['Fare'][i])
    }
    documents.append(document)
    
documents

[{'Survived': '0',
  'Pclass': '3',
  'Name': 'Mr. Owen Harris Braund',
  'Sex': 'male',
  'Age': '22.0',
  'Siblings_Spouses Aboard': '1',
  'Parents_Children Aboard': '0',
  'Fare': '7.25'},
 {'Survived': '1',
  'Pclass': '1',
  'Name': 'Mrs. John Bradley (Florence Briggs Thayer) Cumings',
  'Sex': 'female',
  'Age': '38.0',
  'Siblings_Spouses Aboard': '1',
  'Parents_Children Aboard': '0',
  'Fare': '71.2833'},
 {'Survived': '1',
  'Pclass': '3',
  'Name': 'Miss. Laina Heikkinen',
  'Sex': 'female',
  'Age': '26.0',
  'Siblings_Spouses Aboard': '0',
  'Parents_Children Aboard': '0',
  'Fare': '7.925'},
 {'Survived': '1',
  'Pclass': '1',
  'Name': 'Mrs. Jacques Heath (Lily May Peel) Futrelle',
  'Sex': 'female',
  'Age': '35.0',
  'Siblings_Spouses Aboard': '1',
  'Parents_Children Aboard': '0',
  'Fare': '53.1'},
 {'Survived': '0',
  'Pclass': '3',
  'Name': 'Mr. William Henry Allen',
  'Sex': 'male',
  'Age': '35.0',
  'Siblings_Spouses Aboard': '0',
  'Parents_Children Aboard': 

In [190]:
# Another insert_many method
db.titanic.insert_many(documents)

BulkWriteError: batch op errors occurred